<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interactive-chatbot" data-toc-modified-id="Interactive-chatbot-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interactive chatbot</a></span></li><li><span><a href="#Import-required-libraries" data-toc-modified-id="Import-required-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import required libraries</a></span></li><li><span><a href="#Load-corpus" data-toc-modified-id="Load-corpus-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load corpus</a></span></li><li><span><a href="#Study-corpus" data-toc-modified-id="Study-corpus-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Study corpus</a></span></li><li><span><a href="#Define-functions" data-toc-modified-id="Define-functions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define functions</a></span><ul class="toc-item"><li><span><a href="#Function:-Get-list-of-all-characters-in-a-corpus" data-toc-modified-id="Function:-Get-list-of-all-characters-in-a-corpus-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Function: Get list of all characters in a corpus</a></span></li><li><span><a href="#Function:-Pre-process-features" data-toc-modified-id="Function:-Pre-process-features-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Function: Pre-process features</a></span></li></ul></li><li><span><a href="#Generate-training-set---Features-and-labels" data-toc-modified-id="Generate-training-set---Features-and-labels-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Generate training set - Features and labels</a></span><ul class="toc-item"><li><span><a href="#Create-corpus_docs-and-corpus_tags" data-toc-modified-id="Create-corpus_docs-and-corpus_tags-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Create corpus_docs and corpus_tags</a></span></li><li><span><a href="#Pre-process-features" data-toc-modified-id="Pre-process-features-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Pre-process features</a></span></li><li><span><a href="#Generate-BBOW" data-toc-modified-id="Generate-BBOW-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Generate BBOW</a></span></li><li><span><a href="#Create-X_train-and-y_train" data-toc-modified-id="Create-X_train-and-y_train-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Create X_train and y_train</a></span></li></ul></li><li><span><a href="#Model---Define,-compile-and-train" data-toc-modified-id="Model---Define,-compile-and-train-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Model - Define, compile and train</a></span><ul class="toc-item"><li><span><a href="#Define-model" data-toc-modified-id="Define-model-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Define model</a></span></li><li><span><a href="#Compile-model" data-toc-modified-id="Compile-model-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Compile model</a></span></li><li><span><a href="#Fit-and-evaluate-model" data-toc-modified-id="Fit-and-evaluate-model-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Fit and evaluate model</a></span></li></ul></li><li><span><a href="#Define-chat-function" data-toc-modified-id="Define-chat-function-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Define chat function</a></span></li><li><span><a href="#Execute-Chat" data-toc-modified-id="Execute-Chat-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Execute Chat</a></span><ul class="toc-item"><li><span><a href="#Start-->-Intro-->-Quit" data-toc-modified-id="Start-->-Intro-->-Quit-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Start -&gt; Intro -&gt; Quit</a></span></li><li><span><a href="#Multiple-unrecognized-queries" data-toc-modified-id="Multiple-unrecognized-queries-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Multiple unrecognized queries</a></span></li><li><span><a href="#Normal-query-flow" data-toc-modified-id="Normal-query-flow-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Normal query flow</a></span></li></ul></li></ul></div>

# Interactive chatbot

Problem Statement: Build a python based interactive semi-rule based chatbot.

# Import required libraries

In [1]:
import json

import numpy as np 
import pandas as pd
import nltk
import re

from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense, Activation
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

from tensorflow.keras import optimizers

# Load corpus

In [2]:
with open("GL_Bot.json") as file:
    corpus = json.load(file)

# Study corpus

In [3]:
# Get list of keys in corpus['intents']
corpus['intents'][0].keys()

dict_keys(['tag', 'patterns', 'responses', 'context_set'])

In [4]:
# Look at one entry of corpus['intents']
print("Tag:")
print("====")
print(corpus['intents'][0]['tag'])
print()
print("Patterns:")
print("=========")
print(corpus['intents'][0]['patterns'])
print()
print("Responses:")
print("==========")
print(corpus['intents'][0]['responses'])

Tag:
====
Intro

Patterns:
['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time', 'i need help']

Responses:
['Hello! how can i help you?']


In [5]:
print("Number of unique tags in corpus is %d" %(len(corpus['intents'])))
print()
print("List of unique tags in corpus:")
print("==============================")
print([intent['tag'] for intent in corpus['intents']])
print()
print("Total number of unique patterns in corpus is %d" \
      %(sum([len(intent['patterns']) for intent in corpus['intents']])))

Number of unique tags in corpus is 8

List of unique tags in corpus:
['Intro', 'Exit', 'Olympus', 'SL', 'NN', 'Bot', 'Profane', 'Ticket']

Total number of unique patterns in corpus is 168


In [6]:
# Get some statistics for each tag
cols_list = ['Tag', 'Number of unique patterns', 'Response']
corpus_df = pd.DataFrame(columns = cols_list)
for intent in corpus['intents']:
    entry_dict = {'Tag': intent['tag'],
                  'Number of unique patterns': len(intent['patterns']),
                  'Response': intent['responses'][0]
                 }
    corpus_df = corpus_df.append(entry_dict, ignore_index = True)
display(corpus_df)

,Tag,Number of unique patterns,Response
0,Intro,21,Hello! how can i help you?
1,Exit,20,"I hope I was able to assist you, Good Bye!"
2,Olympus,14,Please refer to this link: https://www.greatle...
3,SL,36,Please refer to this link: https://en.wikipedi...
4,NN,44,Please refer to this link: https://en.wikipedi...
5,Bot,10,I am your virtual learning assistant
6,Profane,9,Please use respectful words!
7,Ticket,14,A ticket has been filed. Please expect a respo...


# Define functions

## Function: Get list of all characters in a corpus

In [7]:
def get_char_list(corpus):
    
    '''
    Get a list of lexicographically sorted characters present in corpus. Each document in corpus
    will be converted to lower-case before generating this list
    
    Arguments:
        corpus - Input corpus from which list of characters has to be extracted.
    
    Returns: 
        List of all characters in corpus sorted lexicographically.
    '''
    
    all_char_pattern = re.compile(r'.') # Match all characters in corpus except newline
    all_char_list = [] # Placeholder to store all characters in corpus

    for doc in corpus: # Loop through all documents in corpus and extract all characters
        doc = doc.lower() # Convert document to lower-case
        matches = all_char_pattern.finditer(doc)
        for mat in matches:
            all_char_list.append(mat.group(0)) # Append characters from current document

    # Get unique characters, convert to list and sort.
    uniq_char_list = sorted(list(set(all_char_list)))
    
    print()
    print("Number of unique characters is %d" %len(uniq_char_list))
    print()
    print("List of unique characters:")
    print("==========================")
    print("{}".format(uniq_char_list))

    return uniq_char_list

## Function: Pre-process features

In [8]:
def preprocess_feat(corpus, stemmer):
    
    '''
    Pre-process each document in corpus. Includes the following:
        1. Remove html tags
        2. Delete single-quote characters
        3. Replace miscellaneous characters with whitespace
        4. Split document into words, perform stemming and merge stemmed words back into document
    
    Arguments:
        corpus - Input corpus.
        stemmer - Stemmer object to use for performing stemming
    
    Returns: 
        Pre-processed corpus.    
    '''

    corpus_docs_processed = [] # Placeholder to store processed documents from corpus
    for doc in corpus: # Loop through all documents in corpus
        # Convert document to all lower-case
        doc_proc = doc.lower()

        # Search for html tags in the current document (if any) and replace them with space
        html_tag_patt = re.compile('<.*?>')
        doc_proc = re.sub(html_tag_patt, ' ', doc_proc)

        # Search for quote character (if any) and delete it
        # This will make words like don't = dont and won't = wont
        pattern_1 = re.compile(r'[\']')
        doc_proc = re.sub(pattern_1, '', doc_proc)

        # Search for miscellaneous characters (if any) and replace them with whitespace
        # Standard symbols in QWErtY keyboard to be replaced with whitespace
        pattern_2 = re.compile(r'[`|~|!|@|#|$|%|^|&|*|(|)|\-|_|=|+|\[|\]|{|}|\|;|:|"|,|.|/|<|>|?]')
        doc_proc = re.sub(pattern_2, ' ', doc_proc)

        # Split document into list of words, perform stemming on each word
        words = [stemmer.stem(word) for word in doc_proc.split()]  
        # Concatenate words back into a document
        doc_proc = ''
        for word in words:
            doc_proc = doc_proc + word + ' '

        # Append current processed document to corpus_docs_processed
        corpus_docs_processed.append(doc_proc)

    return corpus_docs_processed

# Generate training set - Features and labels

## Create corpus_docs and corpus_tags

In [9]:
corpus_docs = [] # Placeholder to store all documents of the corpus for training set
corpus_tags = [] # Placeholder to store all tags of the corpus for training set
for intent in corpus['intents']: # Loop through all intents in corpus
    for pattern in intent['patterns']: # Loop through all patterns in each intent
        corpus_docs.append(pattern) # Append current pattern to corpus_docs
        corpus_tags.append(intent['tag']) # Append current tag to corpus_tags
unique_tags = sorted(list(set(corpus_tags))) # Get list of unique tags

print("Number of documents in corpus is %d" %len(corpus_docs))
print("Number of unique tags in corpus is %d" %len(unique_tags))
print()
print("List of tags:")
print("=============")
print("{}".format(unique_tags))

# Get list of unique characters in corpus_docs
uniq_char_list = get_char_list(corpus_docs)

Number of documents in corpus is 168
Number of unique tags in corpus is 8

List of tags:
['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']

Number of unique characters is 27

List of unique characters:
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Pre-process features

In [10]:
stemmer = PorterStemmer() # Instantiate Porter Stemmer
corpus_docs_processed = preprocess_feat(corpus_docs, stemmer)

In [11]:
# Check corpus_docs_processed
print(corpus_docs[0:10]) # First 10 documents of original corpus
print()
print(corpus_docs_processed[0:10]) # First 10 documents of preprocessed corpus

['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from']

['hi ', 'how are you ', 'is anyon there ', 'hello ', 'what up ', 'hey ', 'yo ', 'listen ', 'pleas help me ', 'i am learner from ']


## Generate BBOW

In [12]:
# Binary BOW count vectorizer
vect_bbow = CountVectorizer(binary = True) 
X_bbow = vect_bbow.fit_transform(corpus_docs_processed)
feat_names_bbow = vect_bbow.get_feature_names()

print("Number of features: {}".format(len(feat_names_bbow)))
print()
print("First 20 features:\n{}".format(feat_names_bbow[:20]))
print()
print("Shape of X_bbow is {}".format(X_bbow.shape))

Number of features: 201

First 20 features:
['abl', 'about', 'access', 'activ', 'ada', 'adam', 'ai', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyon', 'are', 'artifici', 'back', 'backprop', 'backward', 'bad', 'bag']

Shape of X_bbow is (168, 201)


## Create X_train and y_train

In [13]:
X_train = X_bbow.toarray() # Convert sparse bbow array to normal numpy array
y_train = np.array(pd.get_dummies(corpus_tags)) # Peform One-hot encoding on corpus_tags
print("Type of X_train is {}".format(type(X_train)))
print("Type of y_train is {}".format(type(y_train)))
print()
print("Length of X_train is {}".format(X_train.shape))
print("Length of y_train is {}".format(y_train.shape))

Type of X_train is <class 'numpy.ndarray'>
Type of y_train is <class 'numpy.ndarray'>

Length of X_train is (168, 201)
Length of y_train is (168, 8)


# Model - Define, compile and train

## Define model

In [14]:
# Build a simple NN model to fit the above training set
# Input -> Dense(64, ReLu) -> Dropout(0.5) -> Dense(32, ReLu) -> Dropout(0.5) -> Dense(8, softmax)
model = Sequential()
model.add(Dense(64, input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                12928     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
Total params: 15,272
Trainable params: 15,272
Non-trainable params: 0
_________________________________________________________________


## Compile model

In [15]:
# Define Optimizer
opt = optimizers.Adam() 
# Define Loss = CategoricalCrossEntropy
cce = keras.losses.CategoricalCrossentropy() 
# Define Metric = Categorical Accuracy
met = keras.metrics.CategoricalAccuracy()

model.compile(optimizer = opt, loss = cce, metrics = met)

## Fit and evaluate model

In [16]:
hist = model.fit(X_train, y_train, epochs = 200, verbose = 0)
acc = model.evaluate(X_train, y_train, return_dict = True, verbose = 0)['categorical_accuracy']
print("Achieved accuracy on training set is %0.2f %%" %(100 * acc))

Achieved accuracy on training set is 100.00 %


# Define chat function

In [17]:
def chat():
    
    '''
    Function to execute chat
    '''

    # Define a few response strings (not present in the corpus)
    stmt_1 = "Bot: Chat started: To exit the chat, type quit"
    stmt_2 = "Bot: Thank you. Have a good day!"
    stmt_3 = "\nBot: Is there anything else I can help you with? If not, type quit to exit."
    stmt_4 = "Bot: Sorry, I could not understand your question."
    stmt_5 = " Please rephrase your question."    
    stmt_6 = "Bot: You have exceeded the maximum number of attempts."
    stmt_7 = " A ticket has been filed. Please expect a response within 24 hours."    
    
    print(stmt_1) # Statment initializing chat
    print()
    
    # Initialize counter to keep track of number of questions that are not understood
    not_understood_count = 0
    
    while True: # Loop infinitely unless user requests to exit
        inp = input("\nYou: ") # Get input from user
        if (inp.lower() == "quit"): # Exit chat if user types 'quit'
            print(stmt_2)
            break 
        else: # If user input is not 'quit'
            # Pre-process and vectorize input           
            inp_arr = vect_bbow.transform(preprocess_feat([inp], stemmer)).toarray()
            # Check if input has any words from the trained vocabulary
            num_words_vocab = np.sum(inp_arr[0] == 1)
            # If input has no words from vocab, ask user to repeat question
            # If this happens thrice in a row, file a ticket
            if (num_words_vocab == 0):
                not_understood_count += 1
                if (not_understood_count < 3):
                    print(stmt_4 + stmt_5)
                else: # If question has not been understood three times, file a ticket
                    not_understood_count = 0 # Reset not_understood_count
                    print(stmt_6 + stmt_7)
                    print(stmt_3)
            else: # Input has some words from vocab
                # Reset not_understood_count if input has some word from vocabulary
                not_understood_count = 0 
                
                model_out = model.predict(inp_arr) # Get model prediction
                tag_index = np.argmax(model_out) # Get index of predicted tag
                tag = unique_tags[tag_index] # Extract predicted tag
                
                # Extract responses corresponding to predicted tag from corpus
                for intent in corpus['intents']:
                    if intent['tag'] == tag:
                        responses = intent['responses']
                print("Bot: " + np.random.choice(responses))
                if (tag.lower() in ["olympus", 'nn', 'sl', 'ticket']):
                    print(stmt_3)
                if (tag.lower() == 'exit'):
                    break

# Execute Chat

## Start -> Intro -> Quit

In [18]:
chat()

Bot: Chat started: To exit the chat, type quit


You: hi
Bot: Hello! how can i help you?

You: quit
Bot: Thank you. Have a good day!


## Multiple unrecognized queries

In [19]:
chat()

Bot: Chat started: To exit the chat, type quit


You: hello
Bot: Hello! how can i help you?

You: saldkfjalsj
Bot: Sorry, I could not understand your question. Please rephrase your question.

You: oiqwupiue
Bot: Sorry, I could not understand your question. Please rephrase your question.

You: lkjelw;rkjq
Bot: You have exceeded the maximum number of attempts. A ticket has been filed. Please expect a response within 24 hours.

Bot: Is there anything else I can help you with? If not, type quit to exit.

You: quit
Bot: Thank you. Have a good day!


## Normal query flow

In [20]:
chat()

Bot: Chat started: To exit the chat, type quit


You: hello
Bot: Hello! how can i help you?

You: please tell me about yourselves
Bot: I am your virtual learning assistant

You: I want to know about clustering
Bot: Please refer to this link: https://en.wikipedia.org/wiki/Machine_learning

Bot: Is there anything else I can help you with? If not, type quit to exit.

You: Can you also tell me about neural networks?
Bot: Please refer to this link: https://en.wikipedia.org/wiki/Neural_network

Bot: Is there anything else I can help you with? If not, type quit to exit.

You: thank you
Bot: I hope I was able to assist you, Good Bye!
